In [0]:
# python package to allow for dotenv in colab
!pip install colab-env -qU
from colab_env import envvar_handler

In [0]:
# install pre-trained model
!pip install vaderSentiment

In [0]:
import os
from dotenv import load_dotenv
load_dotenv('vars.env')
import psycopg2

# dotenv variables to access databasse
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')
DB_HOST = os.getenv('DB_HOST')

# set up postgres connection
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER,
                        password=DB_PASS, host=DB_HOST)
cursor = conn.cursor()

In [0]:
# Find all entries without scores
query = '''
SELECT comment_id, comment_text 
FROM salty_db_2
WHERE salty_comment_score_pos IS NULL
'''

cursor.execute(query)
records = cursor.fetchall()

# will fail if no entries are found
print(records[0])

In [0]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
# get all records without scores
for record in records:
  # score them
  neg = analyzer.polarity_scores(record[1])['neg']
  pos = analyzer.polarity_scores(record[1])['pos']
  # update database with scores
  query = f'''
    UPDATE salty_db_2
    SET salty_comment_score_pos = {pos}, salty_comment_score_neg = {neg}
    WHERE comment_id = {record[0]}
  '''
  cursor.execute(query)
conn.commit()

In [0]:
# close postgres connection
conn.close()